<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению. Сессия № 2
</center>
Автор материала: программист-исследователь Mail.ru Group, старший преподаватель Факультета Компьютерных Наук ВШЭ Юрий Кашницкий. Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Домашнее задание № 8
## <center> Vowpal Wabbit в задаче прогнозирования популярности статьи на хабре

В этом задании надо побить бенчмарк в [соревновании](https://www.kaggle.com/c/habr-num-bookmarks) на Kaggle Inclass. Как это делать – ограничений нет (кроме, конечно, ручной разметки), прочитать правила можно [тут](https://www.kaggle.com/c/habr-num-bookmarks/rules). Ниже описаны инструкции, как это сделать с Vowpal Wabbit.

Дедлайн: 31 октября 23:59 UTC +3. Решение надо будет загрузить по [ссылке](https://www.dropbox.com/request/g5WOPrxwvcYwADZCuoY7). В этом соревновании нет задачи победить. Цель – побить бенчмарк и продвинуться в [соревновании](https://mlcourse.arktur.io) по прогнозу популярности статьи на Medium. 

In [1]:
import numpy as np
import pandas as pd
import json
from datetime import datetime
from tqdm import tqdm_notebook
from sklearn.metrics import mean_absolute_error

Посмотрим на одну из строчек в JSON-файле: считаем ее с помощью библиотеки json. Эта строчка соответствует [7-ой статье](https://habrahabr.ru/post/7/) на Хабре.

In [2]:
!head -1 ../../data/train.json > ../../data/train1.json

"head" не является внутренней или внешней
командой, исполняемой программой или пакетным файлом.


In [3]:
with open('../../data/train1.json') as inp_json:
    first_json = json.load(inp_json)

In [4]:
first_json.keys()

dict_keys(['_id', '_timestamp', 'url', 'domain', 'published', 'title', 'content', 'polling', 'post_id', 'flags', 'hubs', 'flow', 'tags', 'author', 'link_tags', 'meta_tags'])

Видим 16 полей, перечислим некоторые из них:
- _id, url - URL статьи
- published – время публикации статьи
- domain – сайт (например, habrahahbr.ru или geektimes.ru)
- title – название статьи
- content – текст статьи
- hubs - перечисление хабов, к которым относится статья
- tags – теги статьи
- author – автор статьи, его ник и ссылка на профиль

In [5]:
first_json['_id']

'https://habrahabr.ru/post/7/'

In [6]:
first_json['_timestamp']

1493192186.0903192

In [7]:
first_json['url']

'https://habrahabr.ru/post/7/'

In [8]:
first_json['domain']

'habrahabr.ru'

In [9]:
first_json['published']

{'$date': '2006-07-15T01:48:00.000Z'}

In [10]:
first_json['title']

'Самопроизвольное разлогинивание'

In [11]:
first_json['content']

'У меня такое ощущение, что logout время от времени происходит самопроизвольно, несмотря на то, что чекбокс про логине включен.<br>\r\n<br>\r\nВозможно, это происходит при смене IP-адреса, но я не уверен.'

In [12]:
first_json['polling']

In [13]:
first_json['post_id']

7

In [14]:
first_json['flags']

[]

In [15]:
first_json['hubs']

[{'id': 'hub/habr',
  'title': 'Хабрахабр',
  'url': 'https://habrahabr.ru/hub/habr/'}]

In [16]:
first_json['flow']

In [17]:
first_json['tags']

['логин', 'login']

In [18]:
first_json['author']

{'name': 'Павел Титов',
 'nickname': '@ptitov',
 'url': 'https://habrahabr.ru/users/ptitov'}

In [19]:
first_json['link_tags']

{'alternate': 'https://habrahabr.ru/rss/post/7/',
 'apple-touch-icon-precomposed': '/images/favicons/apple-touch-icon-152x152.png',
 'canonical': 'https://habrahabr.ru/post/7/',
 'icon': '/images/favicons/favicon-16x16.png',
 'image_src': 'https://habrahabr.ru/i/habralogo.jpg',
 'stylesheet': 'https://habracdn.net/habr/styles/1493134745/_build/global_main.css'}

In [20]:
first_json['meta_tags']

{'al:android:app_name': 'Habrahabr',
 'al:android:package': 'ru.habrahabr',
 'al:android:url': 'habrahabr://post/7',
 'al:windows_phone:app_id': '460a6bd6-8955-470f-935e-9ea1726a6060',
 'al:windows_phone:app_name': 'Habrahabr',
 'al:windows_phone:url': 'habrahabr://post/7',
 'apple-mobile-web-app-title': 'Хабрахабр',
 'application-name': 'Хабрахабр',
 'description': 'У меня такое ощущение, что logout время от времени происходит самопроизвольно, несмотря на то, что чекбокс про логине включен.\r\n\r\nВозможно, это происходит при смене IP-адреса, но я не уверен.',
 'fb:app_id': '444736788986613',
 'keywords': 'логин, login',
 'msapplication-TileColor': '#FFFFFF',
 'msapplication-TileImage': 'mstile-144x144.png',
 'og:description': 'У меня такое ощущение, что logout время от времени происходит самопроизвольно, несмотря на то, что чекбокс про логине включен.  Возможно, это происходит при...',
 'og:image': 'https://habrahabr.ru/i/habralogo.jpg',
 'og:title': 'Самопроизвольное разлогинивание'

Загрузим ответы на обучающей выборке.

In [2]:
train_target = pd.read_csv('../../data/train_target.csv',
                          index_col='url')

In [3]:
train_target.head(10)

,target
url,
https://habrahabr.ru/post/7/,0.693147
https://geektimes.ru/post/11/,1.098612
https://geektimes.ru/post/112/,0.000000
https://geektimes.ru/post/1127/,0.000000
https://geektimes.ru/post/12664/,0.000000
https://geektimes.ru/post/168615/,1.386294
https://habrahabr.ru/post/168617/,3.433987
https://geektimes.ru/post/257768/,4.969813
https://habrahabr.ru/post/168625/,1.609438


In [14]:
train_target['target'][0:8]

url
https://habrahabr.ru/post/7/         0.693147
https://geektimes.ru/post/11/        1.098612
https://geektimes.ru/post/112/       0.000000
https://geektimes.ru/post/1127/      0.000000
https://geektimes.ru/post/12664/     0.000000
https://geektimes.ru/post/168615/    1.386294
https://habrahabr.ru/post/168617/    3.433987
https://geektimes.ru/post/257768/    4.969813
Name: target, dtype: float64

Сформируйте обучающую выборку для Vowpal Wabbit, выберите признаки title, tags, domain, flow, author, и hubs из JSON-файла.
От самого текста для начала просто возьмем его длину: постройте признак content_len – длина текста в миллионах символов.
Также постройте признаки: час и месяц публикации статьи. Еще, конечно же, возьмите ответы на обучающей выборке из `train_target`. Ниже пример того, как могут выглядеть первые две строки нового файла.

In [23]:
!head -2 ../../data/habr_train.vw

0.6931470000000001 |title Самопроизвольное разлогинивание |tags логин login |domain habrahabr.ru |flow None |author @ptitov |hubs Хабрахабр |num content_len:0.0 month:7 hour:1
1.0986120000000001 |title Stand-along cообщества против сообществ в рамках социальных сетей |tags сообщества интернет-сообщество социальные сети нишевой бренд |domain geektimes.ru |flow None |author @AlexBruce |hubs Чёрная дыра |num content_len:0.0 month:7 hour:14


In [5]:
def to_vw_format(document, label=None):
    return str(label) + ' |title ' + str(document['title']).replace(":",'').replace('|','') + \
    ' |tags '+ " ".join(document['tags']).replace(":",'').replace('|','')\
    + ' |domain '+ str(document['domain']).replace(":",'').replace('|','') \
    + ' |flow ' + str(document['flow']).replace(":",'').replace('|','')+ \
    ' |author ' + str(document['author']['nickname']).replace(":",'').replace('|','') + \
    ' |hubs '+ " ".join([hub['title'] for hub in document['hubs']]).replace(":",'').replace('|','') + \
    ' |num content_len:%0.2f' % (len(document['content'])/1000000) + \
    ' month:'+ str(datetime.strptime(document['published']['$date'],'%Y-%m-%dT%H:%M:%S.000Z').date().month) \
    + ' hour:'+ str(datetime.strptime(document['published']['$date'],'%Y-%m-%dT%H:%M:%S.000Z').time().hour) +'\n'

In [8]:
target_num=0
valid_arr=[]
with open('../../data/train.json') as inp_json, \
     open('../../data/habr_train.vw', 'w',encoding='utf8') as out_vw,\
    open('../../data/habr_train_valid.vw', 'w',encoding='utf8') as out_valid:
    for line in tqdm_notebook(inp_json):
        
        data_json = json.loads(line)
        if not target_num%5==0:
            out_vw.write(to_vw_format(data_json,train_target['target'][target_num]))
        else:
            out_valid.write(to_vw_format(data_json,train_target['target'][target_num]))
            valid_arr.append(train_target['target'][target_num])
        target_num+=1
        
       
       

In [19]:
with open('../../data/habr_train.vw','r',encoding='utf8') as f:
    lines = f.readlines()
np.random.shuffle(lines)
with open('../../data/habr_train.vw', "w",encoding='utf8') as f:
    f.writelines(lines)

In [15]:
target_num=0
with open('../../data/train.json') as inp_json, \
     open('../../data/habr_train_final.vw', 'w',encoding='utf8') as out_vw:
    for line in tqdm_notebook(inp_json):
        
        data_json = json.loads(line)
        
        out_vw.write(to_vw_format(data_json,train_target['target'][target_num]))

        target_num+=1
        
       

In [44]:
with open('../../data/habr_train_final.vw','r',encoding='utf8') as f:
    flines = f.readlines()
np.random.shuffle(flines)
with open('../../data/habr_train_final.vw', "w",encoding='utf8') as f:
    f.writelines(flines)

Проделайте все то же с тестовой выборкой, вместо ответов подсовывая что угодно, например, единицы.

In [29]:
arr_id=[]
with open('../../data/test.json') as inp_json, \
     open('../../data/habr_test.vw', 'w',encoding='utf8') as out_vwtest:
    for line in tqdm_notebook(inp_json):
        data_json = json.loads(line)
        arr_id.append(data_json['_id'])
        out_vwtest.write(to_vw_format(data_json,1))
       

In [24]:
!head -2 ../../data/habr_test.vw

1 |title День Пи! |tags Пи Pi |domain geektimes.ru |flow None |author @Timursan |hubs Чёрная дыра |num content_len:0.0 month:3 hour:3
1 |title Скрипт для разбиения образов музыкальных CD на треки и конвертации в формат FLAC |tags bash lossless |domain geektimes.ru |flow None |author @da3mon |hubs Чёрная дыра |num content_len:0.01 month:3 hour:0


Выбор того, как валидировать модель, остается за Вами. Проще всего, конечно, сделать отложенную выборку. Бенчмарк, который Вы видите в соревновании (**vw_baseline.csv**) и который надо побить, получен с Vowpal Wabbit, 3 проходами по выборке (не забываем удалять кэш), биграммами и настроенными гиперпараметрами `bits`, `learning_rate` и `power_t`. 

In [37]:
%%time
!vw -d ../../data/habr_train.vw -k -c -f ../../data/vw_model.vw -b 26 -l 0.52\
 --random_seed=17 --passes 3 --ngram 2 --power_t 0.37


Generating 2-grams for all namespaces.
final_regressor = ../../data/vw_model.vw
Num weight bits = 26
learning rate = 0.52
initial_t = 0
power_t = 0.37
decay_learning_rate = 1
creating cache_file = ../../data/habr_train.vw.cache
Reading datafile = ../../data/habr_train.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
13.421687 13.421687            1            1.0   3.6636   0.0000       49
7.830542 2.239398            2            2.0   1.9459   0.4494       21
5.867716 3.904890            4            4.0   0.0000   0.4252       27
10.127694 14.387671            8            8.0   3.8501   1.0858       21
6.814958 3.502222           16           16.0   3.0445   0.9983       33
4.717495 2.620033           32           32.0   4.5326   3.2502       65
4.119800 3.522104           64           64.0   5.1818   3.3943       31
2.937427 1.755054          128          128.0   2.8

Wall time: 12.8 s


In [38]:
!vw -i ../../data/vw_model.vw -t -d ../../data/habr_train_valid.vw \
 -p ../../data/valid_pred.txt --quiet

In [39]:
with open('../../data/valid_pred.txt') as pred_file:
    test_prediction = [float(label) 
                             for label in pred_file.readlines()]
mean_absolute_error(valid_arr,test_prediction)

0.73865953329166678

In [45]:
%%time
!vw -d ../../data/habr_train_final.vw -k -c -f ../../data/vw_model_final.vw -b 26 -l 0.52\
 --random_seed=17 --passes 3 --ngram 2 --power_t 0.37

Generating 2-grams for all namespaces.
final_regressor = ../../data/vw_model_final.vw
Num weight bits = 26
learning rate = 0.52
initial_t = 0
power_t = 0.37
decay_learning_rate = 1
creating cache_file = ../../data/habr_train_final.vw.cache
Reading datafile = ../../data/habr_train_final.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
29.188919 29.188919            1            1.0   5.4027   0.0000       39
18.092704 6.996489            2            2.0   3.0445   0.3994       31
12.405099 6.717495            4            4.0   3.4965   0.8403       43
8.489064 4.573028            8            8.0   3.4012   0.9968       35
6.181993 3.874923           16           16.0   2.9444   1.7420       37
4.888592 3.595190           32           32.0   2.4849   2.9159       45
3.976806 3.065021           64           64.0   1.0986   1.8357       33
2.840455 1.704103          128   

Wall time: 15.5 s


In [46]:
!vw -i ../../data/vw_model_final.vw -t -d ../../data/habr_test.vw \
 -p ../../data/valid_pred_test.txt --quiet

In [47]:
with open('../../data/valid_pred_test.txt') as pred_file:
    final_prediction = [float(label) 
                             for label in pred_file.readlines()]


In [48]:
len(final_prediction)

52913

In [50]:
sample_sub = pd.read_csv('../../data/sample_submission.csv', 
                         index_col='url')

In [51]:
sample_sub.head()


,target
url,
https://geektimes.ru/post/87455/,11.620054
https://geektimes.ru/post/87452/,4.822528
https://geektimes.ru/post/87459/,0.921104
https://habrahabr.ru/post/87461/,1.632126
https://habrahabr.ru/post/5754/,1.952122


In [52]:
your_submission = sample_sub.copy()
your_submission['target'] = final_prediction
#your_submission=pd.DataFrame(final_prediction,index=arr_id,columns=['target'])

your_submission.to_csv('../../submission12.csv')

Для получения баллов в #mlcourse_open команда (из 1 человека) должна называться в точном соответствии с тем, как оно записано в рейтинге.